In [26]:
from Functions import LoadData
from Functions import TestStrategy
from Functions import Backtesting
import os
import numpy as np
import pandas as pd

In [46]:
    def calculate_harmonic_mean(self, ratios):
        return len(ratios) / np.sum(1.0 / ratios)

In [47]:
def calculate_piotroski_score(self, row):
        score = 0
        if row['Return on Common Equity'] > 0:
            score += 1
        if row['Return on Common Equity'] > row['Return on Common Equity_previous']:
            score += 1
        if row['Operating Margin'] > 0:
            score += 1
        if row['Operating Margin'] > row['Operating Margin_previous']:
            score += 1
        if row['Cash Flow per Share'] > 0:
            score += 1
        if row['Current Ratio (x)'] > row['Current Ratio (x)_previous']:
            score += 1
        if row['Total Debt/Equity (%)'] < row['Total Debt/Equity (%)_previous']:
            score += 1
        if row['Shares Outstanding'] <= row['Shares Outstanding_previous']:
            score += 1
        if row['Price/Earnings'] < 15:
            score += 2
        return score

In [ ]:
def select_top_stocks2(self, universe, decision_date):
        """
        Select the best stocks based on Piotroski scores calculated from the harmonic mean of financial ratios.
        
        Parameters:
        - universe (list): List of tickers in the selected universe (Offensive or Defensive).
        - decision_date (str or pd.Timestamp): Date of the universe decision.
        
        Returns:
        list: The top 8 stocks if the universe is Offensive, or all stocks if Defensive.
        """
        decision_date = pd.to_datetime(decision_date)
        df = pd.DataFrame()

        for ticker in universe:
            if ticker in self.financials:
                financial_data = self.financials[ticker]
                report_dates = pd.to_datetime(financial_data.columns, format='%Y-%m', errors='coerce')
                
                # Filter valid dates
                valid_dates = [dt for dt in report_dates if dt <= decision_date]
                
                if valid_dates:
                    most_recent_date = max(valid_dates)
                    ratios_data = financial_data[most_recent_date].loc[self.ratios]
                    
                    # Calculate harmonic mean for each ratio
                    harmonic_mean_ratios = self.calculate_harmonic_mean(ratios_data)

                    # Create a dictionary of ratios for Piotroski score calculation
                    ratios_dict = {ratio: ratios_data[ratio] for ratio in self.ratios}

                    # Add previous year ratios if available
                    previous_date = most_recent_date - pd.DateOffset(years=1)
                    if previous_date in report_dates:
                        previous_ratios_data = financial_data[previous_date].loc[self.ratios]
                        ratios_dict['Previous Return on Common Equity'] = previous_ratios_data['Return on Common Equity']
                        ratios_dict['Previous Total Debt/Equity'] = previous_ratios_data['Total Debt/Equity (%)']
                        ratios_dict['Previous Current Ratio'] = previous_ratios_data['Current Ratio (x)']
                        ratios_dict['Previous Shares Outstanding'] = previous_ratios_data.get('Shares Outstanding', 0)
                        ratios_dict['Previous Operating Margin'] = previous_ratios_data['Operating Margin']
                        ratios_dict['Previous Asset Turnover'] = previous_ratios_data.get('Asset Turnover', 0)

                    # Calculate the Piotroski score
                    piotroski_score = self.calculate_piotroski_score(ratios_dict)

                    # Append the score to the DataFrame
                    df = df.append({'Ticker': ticker, 'Harmonic Mean': harmonic_mean_ratios, 'Piotroski Score': piotroski_score}, ignore_index=True)
                else:
                    print(f"No valid financial data found for {ticker} at {decision_date}")

        # Select top 8 stocks based on Piotroski score
        if not df.empty:
            top_stocks = df.nlargest(8, 'Piotroski Score')['Ticker'].tolist()
            return top_stocks
        else:
            print("No sufficient data to select stocks.")
            return []
